<center> 

<h1 style="text-align:center"> Simply Typed Lambda Calculus </h1>
<h2 style="text-align:center"> CSCI7000-11 S23: Principles of Functional Programming </h2>
</center>

Let's implement a version of STLC. The syntax of types is as following:

\\[
\begin{array}{rcll}
A,B & ≔ & Bool & \text{(Booleans)} \\
    & \mid & Int & \text{(integers)} \\
    & \mid & A \rightarrow B & \text{(functions)} \\
\end{array}
\\]


$
\require{color}
\newcommand{\cr}[1]{{\color{\red}{#1}}}
\newcommand{\pair}[2]{\langle #1, #2 \rangle}
\newcommand{\fst}[1]{{\sf fst} ~#1}
\newcommand{\snd}[1]{{\sf snd} ~#1}
\newcommand{\inl}[2]{{\sf inl} ~[#1] ~#2}
\newcommand{\inr}[2]{{\sf inr} ~[#1] ~#2}
\newcommand{\case}[5]{{\sf match} ~#1~ {\sf of} ~{\sf inl}~#2 \Rightarrow #3 ~|~ {\sf inr}~#4 \Rightarrow #5}
\newcommand{\unitv}{{\tt (~)}}
\newcommand{\inferrule}[3]{\displaystyle{\frac{#1}{#2}~~{\small #3}}}
\newcommand{\infrl}[3]{\displaystyle{\frac{#1}{#2}}}
\newcommand{\yb}[1]{\colorbox{yellow}{$#1$}}
\newcommand{\bb}[1]{\colorbox{lightblue}{$#1$}}
$

The syntax of expressions:

\\[
\begin{array}{rcll}
M,N & ≔ & x & \text{(variable)} \\
    & \mid & M~N & \text{(application)} \\
    & \mid & \lambda x:A.M & \text{(abstraction)}\\
    & \mid & 1,2,3,\ldots & \text{(Numbers)}\\
    & \mid & True,False & \text{(Booleans)}\\
    & \mid & e_1 + e_2 & \text{(plus)}\\
    & \mid & e_1 - e_2 & \text{(Minus)}\\
    & \mid & e_1 * e_2 & \text{(Mult)}\\
    & \mid & e_1 = e_2 & \text{(Equality)}\\
    & \mid & \text{if} ~e_1~ \text{then} ~e_2~ \text{else} ~e_3~ & \text{(If-Then-Else)}\\
    & \mid & \text{fix}~e & \text{(Fixpoint operator)}\\
\end{array}
\\]

In [1]:
type typ =
  | TInt
  | TBool
  | TArr of typ * typ

type typ = TInt | TBool | TArr of typ * typ


In [2]:
type exp =
  | Var of string
  | App of exp * exp
  | Lam of string * typ * exp (* \(x:T).e *)
  | IntConst of int
  | True
  | False
  | Plus of exp * exp
  | Minus of exp * exp
  | Mult of exp * exp
  | Eq of exp * exp
  | Ite of exp * exp * exp
  | Fix of exp

type exp =
    Var of string
  | App of exp * exp
  | Lam of string * typ * exp
  | IntConst of int
  | True
  | False
  | Plus of exp * exp
  | Minus of exp * exp
  | Mult of exp * exp
  | Eq of exp * exp
  | Ite of exp * exp * exp
  | Fix of exp


In [3]:
(*
 * \Gamma |- e: T
 * \Gamma = []
          | (x,T):\Gamma,
 *)
module Gamma = struct
  type t = (string*typ) list
end


module Gamma : sig type t = (string * typ) list end


Type rules

\\[
\begin{array}{cc}
\inferrule{\Gamma(x) ~=~ A}{\Gamma \vdash x:A}{(var)} &
\inferrule{\Gamma \vdash M : A \rightarrow B \quad \Gamma \vdash N : A}{\Gamma \vdash M~N : B}{(\rightarrow elim)} &
\inferrule{\Gamma,x:A \vdash M : B}{\Gamma \vdash \lambda x:A.M : A \rightarrow B}{(\rightarrow intro)} \\  
\end{array}
\\]

\\[
\begin{array}{cc}
 \inferrule{}
           {\texttt{True}: \texttt{Bool}}
           {} & 
 \inferrule{}
           {n: \texttt{Int}}
           {} &
 \inferrule{\Gamma \vdash e_1: \texttt{Int} \quad \Gamma \vdash e_2: \texttt{Int} \quad \otimes \in \{+, \times, - \}}
           {e1 \otimes e_2: \texttt{Int}}
           {} \\          
\end{array}
\\]

\\[
\begin{array}{cc}
\inferrule{\Gamma \vdash e_1: A \quad \Gamma \vdash e_2: A \quad A \in \{\texttt{Int}, \texttt{Bool}\}}
           {e1 = e_2: \texttt{Bool}}
           {} &
\inferrule{\Gamma \vdash e_1: \texttt{Bool} \quad \Gamma \vdash e_2: A \quad \Gamma \vdash e_3: A}
           {\texttt{if}\; e_1 \; \texttt{then}\; e_2 \; \texttt{else} \; e_3: \texttt{A}}
           {} \\
\end{array}
\\]

---

\\[
\begin{array}{cc}
\inferrule{\Gamma \vdash e: (A \rightarrow A) \rightarrow A \rightarrow A}
           {\Gamma \vdash \texttt{fix}\;e: A \rightarrow A}
           {} \\
\end{array}
\\]


In [23]:
exception TypeError

(*
 * Int -> (Int -> Int)
 * \(x:Int).\(y:Int). x+y
 *)
let rec type_of (env: Gamma.t) (e: exp) : typ = match e with
  | Var x -> (try List.assoc x env with Not_found -> raise TypeError)
  | Lam(x,t2,e2) -> 
    let env' = (x,t2)::env in
    let t3 = type_of env' e2 in
    (match t3 with 
      | TArr (t31, t32) -> TArr (TArr (t2,t31), t32)
      | _ -> TArr (t2,t3))
  | App (e1,e2) -> 
    (* e1 : t11 -> t12  e : t2 *)
    let t1 = type_of env e1 in
    let t2 = type_of env e2 in
    (match t1 with
    | TArr(t11,t12) when t11 = t2 -> t12
    | _ -> raise TypeError)
  | Plus (e1,e2) | Mult (e1,e2) | Minus (e1,e2) -> 
    let t1 = type_of env e1 in
    let t2 = type_of env e2 in
    (match (t1,t2) with
      | (TInt,TInt) -> TInt
      | _ -> raise TypeError)
  | Eq (e1, e2) -> 
    let t1 = type_of env e1 in
    let t2 = type_of env e2 in
    (match (t1,t2) with
      | (TInt,TInt) -> TBool
      | (TBool,TBool) -> TBool
      | _ -> raise TypeError)
  (* if e1 then e2 else e3 *)
  | Ite (e1,e2,e3) -> 
    let t1 = type_of env e1 in
    let t2 = type_of env e2 in
    let t3 = type_of env e3 in
    (match (t1,t2,t3) with 
      | (TBool, _ , _) when t2 = t3 -> t2
      | _ -> raise TypeError)
  | Fix e -> 
    let t = type_of env e in
    (match t with
     (* (t1 -> t2) -> t3 -> t4  *)
     | TArr (TArr (TArr (t1,t2),t3),t4) when t1 = t2 && t2 = t3 && t3 = t4 -> TArr (t1,t2)
     | _ -> raise TypeError)
  | True | False -> TBool
  | IntConst _ -> TInt

exception TypeError


val type_of : Gamma.t -> exp -> typ = <fun>


In [ ]:
exception NotClosed

type valu = 
 | IntVal of int
 | TrueVal
 | FalseVal
 | LamVal of string * typ * exp

let eval (e:exp) : valu = match e with
  | Var _ -> raise NotClosed
  | _ -> failwith "Not. Impl"
  
type valenv = (string * valu) list

let eval (e:exp) (env: valenv) : valu = 
  | Var x -> (try List.assoc x env with Not_found -> raise NotClosed)
  | _ -> ..

In [26]:
(*
 * myfact : (Int -> Int) -> Int -> Int
 * myfact = \(f: Int -> Int).\(n: Int). if n=0 then 1 else n * f(n-1)
 * (fix myfact) : Int -> Int
 *)
 let else_e = Mult(Var "n", App (Var "f", Minus(Var "n", IntConst 1))) in
 let bool_e = Eq(Var "n", IntConst 0) in
 let ite_e = Ite(bool_e, IntConst 1, else_e) in
 let _ = type_of [("f",TArr(TInt,TInt)); ("n", TInt)] ite_e in
 let inner_lam_e = Lam ("n",TInt,ite_e) in
 let _ = type_of [("f",TArr(TInt,TInt))] inner_lam_e in
 let outer_lam_e = Lam ("f", TArr(TInt, TInt), inner_lam_e) in
 let _ = type_of [] outer_lam_e in
 let fact = Fix outer_lam_e in
 type_of [] fact

- : typ = TArr (TInt, TInt)


In [ ]:
type_of [] 

In [8]:
let rec free_vars (e:exp) : string list = match e with
     | Var x -> [x]
     | Lam (x,e) -> List.filter (fun y -> x <> y) (free_vars e)
     | App (e1,e2) -> (free_vars e1) @ (free_vars e2)

error: compile_error

In [ ]:
let rec subst (e:exp) (y:string) (m:exp) : exp = match e with
  | Var x ->
       if y = x then m (* replace x with m *)
       else e (* variables don't match: leave x alone *)
  | App (e1,e2) -> App (subst e1 y m, subst e2 y m)
  | Lam (x,e) ->
       if y = x then (* don't substitute under the variable binder *)
           Lam(x,e)
       else if not (List.mem x (fvs m)) then (* no need to alpha convert *)
           Lam (x, subst e y m)
       else (* need to alpha convert *)
           let z = fresh () in (* assumed to be "fresh" *)
           let e' = subst e x (Var z) in (* replace x with z in e *)
           Lam (z,subst e' y m) (* substitute for y in the adjusted term, e' *)

In [ ]:
let rec eval (e:exp) : exp = match e with
  | App (e1, e2) -> (match eval e1, eval e2 with
      | Lam (x,e), v -> eval @@ subst e x v
      | _,_ -> App (e1,e2))
  | _ -> e 